# 171 - Square Sum of the Digital Squares

## Problem Statement

For a positive integer $n$, let $f(n)$ be the sum of the squares of the digits (in base $10$) of $n$, e.g.

\begin{align}
f(3) &= 3^2 = 9,\\
f(25) &= 2^2 + 5^2 = 4 + 25 = 29,\\
f(442) &= 4^2 + 4^2 + 2^2 = 16 + 16 + 4 = 36\\
\end{align}

Find the last nine digits of the sum of all $n$, $0 \lt n \lt 10^{20}$, such that $f(n)$ is a perfect square.

## Solution

Given $n$ has at most 20 digits, the maximum value of $f(n)$ is $20 \times 9^2 = 1620$. There are only 40 squares lower than this. For each of those squares, $s$, we can compute all the combinations of 20 squares (including 0) that sum to $s$. This can be achieved with a modified knapsack algorithm. 

For a given combination, we need to find the of the last 9 digits for all the permutations. Let $m_0, m_1, ..., m_9$ represent the number of $0, 1, ..., 9$ in the digit combination. The total number of distinct permutations, $N$, is given by

\begin{equation}
    N = \frac{20!}{m_0!m_1! \cdots m_9!}.
\end{equation}

In each permutation, each of the 20 positions is equally likely to contain each digit (according to its multiplicity). This means that among the $N$ permutations, the number that have the digit $d$ in a specific position is given by 
$$\frac{m_d}{20}N.$$ 
The contribution of all the digits for a specific position is the sum of the individual contributions given by
$$ \frac{N}{20} \sum_0^9 dm_d. $$
Finally, we need to consider the contribution for each of the last 9 positions in the permutation. The $i$-th rightmost contribution needs to be multiplied $10^{i-1}$. Therefore, for the last 9 digits, we need to multiply the contribution by $\sum_{i = 0}^8 10^i = 111111111$. Based on this, we can compute the contribution to the total sum of the combination as
$$ 111111111 \frac{N}{20} \sum_0^9 dm_d. $$

We now have everything we need. For each valid $f(n)$ we compute all the possible digit combinations and sum the contribution from each permutation of the digits. Finally, we apply modulo $10^9$ on the result to extract the last 9 digits.

In [5]:
import numpy as np
from math import factorial
from collections import Counter


def knapsack_combinations_unique(n, max_elements):
    """
    Finds all unique combinations of perfect squares that sum to `n`
    with a maximum of `max_elements` in each combination.
    """
    # List of perfect squares less than or equal to n
    perfect_squares = [i**2 for i in range(1, 10)]

    # DP table to store unique combinations
    dp = [[] for _ in range(n + 1)]
    dp[0] = [[]]  # Base case: one way to make sum 0

    # Fill the DP table
    for square in perfect_squares:
        for target in range(square, n + 1):
            for combination in dp[target - square]:
                # Enforce non-decreasing order
                if len(combination) < max_elements and (not combination or square >= combination[-1]):
                    dp[target].append(combination + [square])

    # Return all unique combinations that sum to n
    return dp[n]


def sum_of_last_9_digits_over_permutations(digits):
    """
    Given a list of 20 digits (0..9, possibly with repeats),
    return the sum of the last 9 digits (as a 9-digit number)
    over all distinct permutations of those 20 digits.
    """

    cnt = Counter(digits)
    digit_sum = sum(digits)

    # Count number of distinct permutations
    n_permutations = factorial(20)
    for digit_count in cnt.values():
        n_permutations //= factorial(digit_count)

    return 111111111 * (n_permutations * digit_sum) // 20

In [6]:
squares = []
i = 1
while i**2 <= 20*81:
    squares.append(i**2)
    i += 1

res = 0
for square in squares:
    combs = knapsack_combinations_unique(square, 20)
    for comb in combs:
        for i in range(len(comb)):
            comb[i] = int(np.sqrt(comb[i]))
        comb = [0] * (20 - len(comb)) + comb
        res += sum_of_last_9_digits_over_permutations(comb)

res % 10**9

142989277